# Fantasy Premier League Moneyball
        

# The plan:

link to trello board:
https://trello.com/b/FC7BUMHM/fpl-moneyball



#### 1. Pull data from fantasy premier league site to create datasets for:
- Club data 
- Player data
- Player history
- Fixture data
- My team data
- Fixtures / gameweek multiplier
- Previous season points (the previous season 'points per appearance has been added as gameweek 0 history)

    
#### 2. Create player score based on:
- v1: total points & total points per million
- v2: predicted points next gameweek / next 3 gameweeks and predicted points per million

    
#### 3. Create Optimal Team function using player score v1 or v2    


#### 4. Figure out a way to make transfer most efficiently to maximise my team score

#### Optimisation constraints:
- My budget (start at £100 million but can go up and down based on player prices changing)
- Fill a team of 15 players consisting of 
        - 2 x GKP
        - 5 x DEF
        - 5 x MID
        - 3 X FWD
- 4 of these players will be subs and do not generate you gamepoints.
- The 'starting 11' Can be of any combination/formation, providing 1 goalkeeper, at least 3 defenders and at least 1 forward are selected at all times.
- Only 1 free transfer can be made each week, further transfers deduct 4 points from your total points


#### We want to Maximise:
- Total points per player
- Player score (points per million x fixture multiplier) x chance to play
- Average points over next 3 gameweeks

#### Caps
- You are given a selection of boosts / advantages (called Caps) which you can play at any time, these are:
        - 2 x Wildcards: Make unlimited free transfers to your team in a gameweek
        - 1 x Free Hit: Same as above, but team reverts back to previous lineup after 1 week
        - Triple Captain: Your captain scores you triple points for the next gameweek
        - Bench Boost: The points from all your subs are included in your total score for the gameweek


## 1. Setting Login Details, Imports and Creating Functions

In [2]:
login = ''
password = ''
my_team_id = '3366741'
free_transfers_available = 1
transfers_made_since_last_deadline = False
new_money_in_bank = 2.7

# Upload_path depending on which computer I'm using
upload_path = r'C:\Users\andre\Documents\GitHub\murry_code\Fantasy Premier League\\' # ACER
# upload_path = r'C:\Users\andrew.morris\Documents\GitHub\murry_code\Fantasy Premier League\\' # HP

## Other teams
# my_team_id = '3366741' # acm
# my_team_id = '6442816' # riiaah 
# my_team_id = '6443347' #amao

print('--- Manual inputs set ---')
print("--- all operations complete ---")

#############
## imports ##
#############

import pandas as pd
from datetime import datetime as dt
import requests
import numpy as np
import time
import math 
import itertools
import statistics

#######################################
### Setting dataframe display limits###
#######################################

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

#########################################################
### Create a login session for my team specific pages ###
#########################################################

session = requests.session()

## Create session with login details for urls which require login ##
## Add pasword and login email
login_url = 'https://users.premierleague.com/accounts/login/'
payload = {'password': password,
           'login': login,
           'redirect_uri': 'https://fantasy.premierleague.com/a/login',
           'app': 'plfpl-web'}
session.post(login_url, data=payload)

print('--- Session created for '+login+ '---')

################
### API URLs ###
################


## api url for the main dataset on player stats 
main_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

## api for my team
team_url = 'https://fantasy.premierleague.com/api/my-team/'+my_team_id+'/'

## api for my team details
my_entry_ses = (session.get('https://fantasy.premierleague.com/api/entry/'+my_team_id+'/'))
my_entry_json = my_entry_ses.json()
current_gameweek = my_entry_json['current_event']+1
print('--- Current gameweek set to '+ str(current_gameweek)+' ---')
# calculating money in bank, if no transfers made since last deadline pull from FPL API, else use 'new_money_in_bank'
money_in_bank = my_entry_json['last_deadline_bank']/10
def money_in_bank_to_use():
    if transfers_made_since_last_deadline == True:
        return new_money_in_bank
    else: 
        return money_in_bank
money_in_bank = money_in_bank_to_use()
print('--- Money In The Bank: £'+str(money_in_bank)+' ---')

## api for fixtures
fix_url = 'https://fantasy.premierleague.com/api/fixtures/'

print('--- API URL Variables Created ---') 

# Export a dataframe to csv
# Pass the dataframe through first then optional filname and export path
def export_to_csv(dataframe, filename = 'FPL_data', path = 'C:\\Python CSV Output\\'):
    print('----------------')
    print('Start csv export')
    filename = 'FPL_' + filename + '-' + str(dt.now().strftime("%Y%m%d-%H%M%S"))
    print('- filename: '+ filename)
    path = path+filename+'.csv'
    print('- save location: ' + path)
    dataframe.to_csv(path, index = False, encoding="utf8")
    print('- successfully exported')
    print('-----------------------')

print("--- 'export_to_csv' function created ---")    

# Calculate the time it takes to run a cell
# enter <start_time = time.time()> at the beginning of a cell then call this function at the end of a cell
def time_taken(start_time):
    start_time = start_time
    seconds_to_run = (time.time() - start_time)
    minutes = str(math.floor(seconds_to_run / 60)) + ' minutes '
    seconds = str(round(seconds_to_run % 60)) + ' seconds '
    print("--- Time taken to run: " + minutes + seconds + " ---")

print("--- 'time_taken' function created ---")  

## Creating a function to make an empty dataframe from any dataframe you pass through the function

def create_empty_df(dataframe_name):
    list_of_col = []
    for col in dataframe_name.columns: 
        list_of_col.append(col) 
    return pd.DataFrame(columns=list_of_col)

print("--- 'create_empty_df' function created ---")


## Each player has an api url with their own history
## This function will iterate through all the players
## By inserting their element (id) into the url and appending to a blank dataframe

def create_player_history_table():
    start_time = time.time()
    print('--- Starting to create player history dataframe ---')
    # creating a blank dataframe from the first players url
    url = 'https://fantasy.premierleague.com/api/element-summary/1/'
    req = requests.get(url)
    json = req.json()
    df = pd.DataFrame(json['history'])
    player_history_df = create_empty_df(df)
    # creating a range containing each player id
    id_range = range(1,len(full_player_df)+1)
    
    for i in id_range:    
        i_url = 'https://fantasy.premierleague.com/api/element-summary/'+str(i)+'/'
        i_req = requests.get(i_url)
        i_json = i_req.json()
        i_df = pd.DataFrame(i_json['history'])
        player_history_df = player_history_df.append(i_df, ignore_index=True)
    return player_history_df

print("--- 'create_player_history_table' function created ---")
print("--- all operations complete ---")

--- Manual inputs set ---
--- all operations complete ---
--- Session created for andrew_c_morris@hotmail.co.uk---
--- Current gameweek set to 12 ---
--- Money In The Bank: £0.5 ---
--- API URL Variables Created ---
--- 'export_to_csv' function created ---
--- 'time_taken' function created ---
--- 'create_empty_df' function created ---
--- 'create_player_history_table' function created ---
--- all operations complete ---


## 2. Creating Core Dataframes 

In [3]:
####################
## CLUB DATAFRAME ##
####################

# Creating a dataframe for the club (team) data

main_req = requests.get(main_url)
main_json = main_req.json()
club_df = pd.DataFrame(main_json['teams'])

print("--- 'club_df' dataframe created ---")    

# club_df.head()

###################################
### PREVIOUS SEASON PLAYER DATA ###
###################################

# Creating a dataframe for the previos season player data
# Several uses, will contain the player score used to determine best team / transfers 

import_previous_season_path = upload_path

prev_season_filename = "FPL_Players_2019_2020_season_v2.csv"
prev_season_player_df = pd.read_csv(import_previous_season_path+prev_season_filename) 

# Removing records where a player did not play last season
prev_season_player_df = prev_season_player_df[prev_season_player_df.appearances > 1]


prev_season_player_df['points_per_gameweek'] = round(prev_season_player_df.total_points / 38,2)
prev_season_player_df['points_per_appearance'] = prev_season_player_df.total_points / prev_season_player_df.appearances 
prev_season_player_df['round'] = 0
prev_season_player_df['opponent_team'] = 'Previous Season'
prev_season_player_df['minutes_per_appearance'] = round(prev_season_player_df.minutes / prev_season_player_df.appearances,0)

print("--- 'prev_season_player_df' dataframe created ---")    

# Creating a single row per player to reflect their season as if it were one gameweek, 
# This will be concatenated later with this current seasons player history

prev_season_player_history_df = prev_season_player_df[['id', 
                                                       'opponent_team', 
                                                       'points_per_appearance', 
                                                       'round', 
                                                       'minutes_per_appearance']].copy()

prev_season_player_history_df['minutes_per_appearance'] = prev_season_player_history_df.minutes_per_appearance.astype(int)

prev_season_player_history_df = prev_season_player_history_df.rename(columns={"id": "element",
                                                                              "points_per_appearance": "total_points",
                                                                              "minutes_per_appearance": "minutes"
                                                                             })    
    
print("--- 'prev_season_player_history_df' dataframe created ---")    


  
### GAMEWEEK ZERO DATAFRAME ###

# This is creating a view of points per appearance for the last season
# Which will be added to the player history as 'gameweek zero' 
# And will act as the points a player has scored in a gameweek in the current season
# The point of this is to enrich player this seasons data, but become less important as data from this season builds up


gameweek_zero_df = prev_season_player_df[['id', 
                                          'opponent_team', 
                                          'points_per_appearance', 
                                          'round', 
                                          'minutes_per_appearance']].copy()

gameweek_zero_df['minutes_per_appearance'] = gameweek_zero_df.minutes_per_appearance.astype(int)

gameweek_zero_df = gameweek_zero_df.rename(columns={"id": "element",
                                                    "points_per_appearance": "total_points",
                                                    "minutes_per_appearance": "minutes"
                                                    })


print("--- 'gameweek_zero_df' dataframe created ---")
# gameweek_zero_df.head()

#####################
#### PLAYER DATA ####
#####################

# Creating full_player_df
# Several uses, will contain the data on individual players such as points and is used to determine best team / transfers

full_player_df = pd.DataFrame(main_json['elements'])         # Player dataframe
position_types_df = pd.DataFrame(main_json['element_types']) # Position dataframe
events_df = pd.DataFrame(main_json['events'])                # Gameweek dataframe

# Updating and creating some of the fields in full player dataframe: 
    # Pulling through position and team name instead of 'id', and changing data types
full_player_df['position_abv'] = full_player_df.element_type.map(position_types_df.set_index('id').singular_name_short)
full_player_df['team_abv'] = full_player_df.team.map(club_df.set_index('id').short_name)
full_player_df['team_and_position'] = + full_player_df['team_abv'] + '-' + full_player_df['position_abv']

    # Pulling through data related to previous season
full_player_df['prev_season_points'] = full_player_df.id.map(prev_season_player_df.set_index('id').total_points).fillna(0.0)
full_player_df['prev_season_points_per_gw'] = full_player_df.id.map(prev_season_player_df.set_index('id').points_per_gameweek).fillna(0.0)
full_player_df['prev_season_minutes'] = full_player_df.id.map(prev_season_player_df.set_index('id').minutes).fillna(0.0)

    # Converting selected_by_percent to float data type
full_player_df['selected_by_percent'] = full_player_df.selected_by_percent.astype(float)

    # Creating chance of playing next round multiplier
full_player_df['nxt_round_play_chance'] = (full_player_df['chance_of_playing_next_round']/100.0).fillna(1.0)
full_player_df['this_round_play_chance'] = (full_player_df['chance_of_playing_this_round']/100.0).fillna(1.0)

    # Creating custom points_per_mil
full_player_df['cost_in_mil'] = full_player_df['now_cost']/10
full_player_df['points_per_mil'] = round(full_player_df['total_points']/full_player_df['cost_in_mil'],2)
full_player_df['prev_season_points_per_mil'] = ((round(full_player_df['prev_season_points']/full_player_df['cost_in_mil'],2))/38)*(current_gameweek-1)
full_player_df['total_points_inc_prv_season'] = full_player_df.total_points + full_player_df.prev_season_points_per_gw
full_player_df['current_gameweek'] = current_gameweek
full_player_df['net_transfers_this_gameweek'] = full_player_df.transfers_in_event - full_player_df.transfers_out_event
full_player_df['net_transfers_this_season'] = full_player_df.transfers_in - full_player_df.transfers_out

print("--- 'full_player_df' dataframe created ---")

### FIXTURE DATA ###
### Creating a dataframe for the fixtures ###
### Primarily used to create a gameweek difficulty multiplier which is applied to the player score

#########################################
#### full fixture dataframe creation ####
#########################################

## Getting fixture data from the api url
fix_req = requests.get(fix_url)

## Converting that data to json then dataframe
fix_json = fix_req.json()
fix_df = pd.DataFrame(fix_json)

# Replacing null game weeks with 99 and converting data type to int, required for matches not yet scheduled
fix_df['event'] = fix_df.event.fillna(99.0).astype(int)

# Adding club names and abbreviations as new columns
fix_df['home_team_full']  = fix_df.team_h.map(club_df.set_index('id').name)
fix_df['home_team_short'] = fix_df.team_h.map(club_df.set_index('id').short_name)
fix_df['away_team_full']  = fix_df.team_a.map(club_df.set_index('id').name)
fix_df['away_team_short'] = fix_df.team_a.map(club_df.set_index('id').short_name)

# Adding Home/Away Strengths to fix_df then blending the home and away strengths, 
# As there are no crowds this year the home advantage might be underplayed
# Logic which utilises the home and away scores separately is saved under random at the end of the code

# Keep an eye on this to see if they flip around at any point
    # Home team strength scores
fix_df['h_team_str_overall_home'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_away)
fix_df['h_team_str_att_home'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_away)
fix_df['h_team_str_def_home'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_away)
fix_df['h_team_str_overall_away'] = fix_df.team_h.map(club_df.set_index('id').strength_overall_home)
fix_df['h_team_str_att_away'] = fix_df.team_h.map(club_df.set_index('id').strength_attack_home)
fix_df['h_team_str_def_away'] = fix_df.team_h.map(club_df.set_index('id').strength_defence_home)

    # Blending home and away strength scores for the home team
fix_df['h_team_str_overall']  = fix_df[['h_team_str_overall_home','h_team_str_overall_away']].mean(axis=1)
fix_df['h_team_str_att'] = fix_df[['h_team_str_att_home','h_team_str_att_away']].mean(axis=1)
fix_df['h_team_str_def'] = fix_df[['h_team_str_def_home','h_team_str_def_away']].mean(axis=1)

    # Away team strength scores
fix_df['a_team_str_overall_away'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_home)
fix_df['a_team_str_att_away'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_home)
fix_df['a_team_str_def_away'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_home)
fix_df['a_team_str_overall_home'] = fix_df.team_a.map(club_df.set_index('id').strength_overall_away)
fix_df['a_team_str_att_home'] = fix_df.team_a.map(club_df.set_index('id').strength_attack_away)
fix_df['a_team_str_def_home'] = fix_df.team_a.map(club_df.set_index('id').strength_defence_away)
    
    # Blending home and away strength scores for the away team
fix_df['a_team_str_overall']  = fix_df[['a_team_str_overall_home','a_team_str_overall_away']].mean(axis=1)
fix_df['a_team_str_att'] = fix_df[['a_team_str_att_home','a_team_str_att_away']].mean(axis=1)
fix_df['a_team_str_def'] = fix_df[['a_team_str_def_home','a_team_str_def_away']].mean(axis=1)

# Creating home/away position specific multipliers
    # Basic logic is:
        # For goalkeepers or defenders - home team defense score / away team attack score
        # For midfielders - home team overall score / away team overall score
        # For forwards - home team attack score / away team defense score
    
    # Home team multipliers
fix_df['home_GKP_DEF_multi'] = fix_df['h_team_str_def'] / fix_df['a_team_str_att']
fix_df['home_MID_multi'] = fix_df['h_team_str_overall'] / fix_df['a_team_str_overall']
fix_df['home_FWD_multi'] = fix_df['h_team_str_att'] / fix_df['a_team_str_def']

    # Away team multiplier
fix_df['away_GKP_DEF_multi'] = fix_df['a_team_str_def'] / fix_df['h_team_str_att']
fix_df['away_MID_multi'] = fix_df['a_team_str_overall'] / fix_df['h_team_str_overall']
fix_df['away_FWD_multi'] = fix_df['a_team_str_att'] / fix_df['h_team_str_def']

    # Renaming 'event' to 'gameweek'
fix_df = fix_df.rename(columns={"event": "gameweek"})

print("--- 'fix_df' dataframe created ---")

####################################
### GAMES PLAYED BY EACH CLUB DF ###
####################################

### creating a dataframe of all historic fixtures to provide a number of games played by each team 

club_games_played_df = fix_df[fix_df.gameweek < current_gameweek]
club_games_played_df = pd.concat([club_games_played_df[['gameweek','team_a']].rename(columns={'team_a':'team_id'}),
                                  club_games_played_df[['gameweek','team_h']].rename(columns={'team_h':'team_id'})]) 
club_games_played_df = club_games_played_df.merge(club_df[['id', 'short_name']],
                                                  how = 'left', left_on = ['team_id'], right_on = ['id']).copy()
club_games_played_df = club_games_played_df.drop(['id'], axis=1)
club_games_played_df = club_games_played_df.groupby(['short_name']).size().reset_index(name='counts')
club_games_played_df = club_games_played_df.rename(columns={'short_name':'team_abv','counts':'games_played'})

#######################################################
### SLIM FIXTURES DATAFRAME AND GAMEWEEK MULTIPLIER ###
#######################################################

# Creating a slim version of the fixtures dataframe with a row per position, per team, per fixture  
# Firstly done by creating a dataframe for each position for home and away teams then concatenating all the dataframes

###################################
### Goalkeepers playing at home ###
###################################

GKP_home_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_h', 'home_team_short', 'away_team_short', 'team_h_difficulty', 'home_GKP_DEF_multi'
                         ]].copy()
GKP_home_fix_df['position_abv'],GKP_home_fix_df['home_or_away'] = ['GKP','home']
GKP_home_fix_df = GKP_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_GKP_DEF_multi':'match_multiplier'
                                                 })

#################################
### Defenders playing at home ###
#################################

DEF_home_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_h', 'home_team_short', 'away_team_short', 'team_h_difficulty', 'home_GKP_DEF_multi'
                         ]].copy()
DEF_home_fix_df['position_abv'],DEF_home_fix_df['home_or_away'] = ['DEF','home']
DEF_home_fix_df = DEF_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_GKP_DEF_multi':'match_multiplier'
                                                 })

###################################
### Midfielders playing at home ###
###################################

MID_home_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_h', 'home_team_short','away_team_short', 'team_h_difficulty','home_MID_multi'
                         ]].copy()
MID_home_fix_df['position_abv'],MID_home_fix_df['home_or_away'] = ['MID','home']
MID_home_fix_df = MID_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_MID_multi':'match_multiplier',
                                                 })

################################
### Forwards playing at home ###
################################

FWD_home_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_h', 'home_team_short', 'away_team_short', 'team_h_difficulty', 'home_FWD_multi'
                        ]].copy()
FWD_home_fix_df['position_abv'],FWD_home_fix_df['home_or_away'] = ['FWD','home']
FWD_home_fix_df = FWD_home_fix_df.rename(columns={'team_h':'team_id',
                                                  'home_team_short':'team_abv',
                                                  'away_team_short':'opponent_abv',
                                                  'team_h_difficulty':'match_difficulty',
                                                  'home_FWD_multi':'match_multiplier'
                                                 })

################################
### Goalkeepers playing away ###
################################

GKP_away_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_a', 'away_team_short', 'home_team_short', 'team_a_difficulty', 'away_GKP_DEF_multi'
                         ]].copy()
GKP_away_fix_df['position_abv'],GKP_away_fix_df['home_or_away'] = ['GKP','away']
GKP_away_fix_df = GKP_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_GKP_DEF_multi':'match_multiplier'
                                                 })

##############################
### Defenders playing away ###
##############################

DEF_away_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_a', 'away_team_short', 'home_team_short', 'team_a_difficulty', 'away_GKP_DEF_multi'
                         ]].copy()
DEF_away_fix_df['position_abv'],DEF_away_fix_df['home_or_away'] = ['DEF','away']
DEF_away_fix_df = DEF_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_GKP_DEF_multi':'match_multiplier'
                                                 })

################################
### Midfielders playing away ###
################################

MID_away_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_a', 'away_team_short', 'home_team_short', 'team_a_difficulty', 'away_MID_multi'
                         ]].copy()
MID_away_fix_df['position_abv'],MID_away_fix_df['home_or_away'] = ['MID','away']
MID_away_fix_df = MID_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_MID_multi':'match_multiplier'
                                                 })

#############################
### Forwards playing away ###
#############################

FWD_away_fix_df = fix_df[[
    'gameweek', 'kickoff_time', 'team_a', 'away_team_short', 'home_team_short', 'team_a_difficulty','away_FWD_multi'
                         ]].copy()
FWD_away_fix_df['position_abv'],FWD_away_fix_df['home_or_away'] = ['FWD','away']
FWD_away_fix_df = FWD_away_fix_df.rename(columns={'team_a':'team_id',
                                                  'away_team_short':'team_abv',
                                                  'home_team_short':'opponent_abv',
                                                  'team_a_difficulty':'match_difficulty',
                                                  'away_FWD_multi':'match_multiplier'
                                                 })

## Concatenating all the above dataframes
slim_fix_df = pd.concat([GKP_home_fix_df,
                         DEF_home_fix_df,
                         MID_home_fix_df,
                         FWD_home_fix_df,
                         GKP_away_fix_df,
                         DEF_away_fix_df,
                         MID_away_fix_df,
                         FWD_away_fix_df
                        ])

# Adding boolean fields for identifying current, historic, future and next 5 games weeks 
slim_fix_df['is_current_gameweek'   ] = slim_fix_df.gameweek == current_gameweek 
slim_fix_df['is_historic_gameweek'  ] = slim_fix_df.gameweek <  current_gameweek
slim_fix_df['is_future_gameweek'    ] = slim_fix_df.gameweek >  current_gameweek
slim_fix_df['is_in_next_3_gameweeks'] = slim_fix_df['gameweek'].apply(lambda a: a in range(current_gameweek, current_gameweek+3))
slim_fix_df['is_in_next_5_gameweeks'] = slim_fix_df['gameweek'].apply(lambda a: a in range(current_gameweek, current_gameweek+5))

# Combining team_abv and position_abv to use for mapping 
slim_fix_df['team_and_position'] = slim_fix_df.team_abv + '-' + slim_fix_df.position_abv

# Sorting by kickoff datetime 
slim_fix_df = slim_fix_df.sort_values(by=['kickoff_time']).reset_index(drop = True)

print("--- 'slim_fix_df' dataframe created ---")

##############################################
### Creating Gameweek Multiplier Dataframe ###
##############################################

# ##########################################
# ### Next 5 Gameweek Average Multiplier ###
# ##########################################

# # Not using this at the moment

# Creating a dataframe of just the next 5 gameweeks with columns 
next_5_gameweeks_multi_df = slim_fix_df[slim_fix_df.is_in_next_5_gameweeks == True]

# Creating a Pivot table of next 5 gameweeks with 'team_and_position' & average 'match_multiplier' 
next_5_gameweeks_multi_pivot = next_5_gameweeks_multi_df.pivot_table(index=['team_and_position'],
                                                          values=['match_multiplier'], 
                                                          aggfunc=np.mean)                                                  
# Transforming the pivot table back into a dataframe
next_5_gameweeks_multi_df = pd.DataFrame(next_5_gameweeks_multi_pivot.to_records()).rename(columns={"match_multiplier": "nxt_5_gw_avg_multi"})

print("--- 'next_5_gameweeks_multi_df' dataframe created ---")
# print(next_5_gameweeks_multi_df.head())

##########################################
### Next 3 Gameweek Average Multiplier ###
##########################################

# Creating a dataframe of just the next 3 gameweeks with columns 
next_3_gameweeks_multi_df = slim_fix_df[slim_fix_df.is_in_next_3_gameweeks == True]

# Creating a Pivot table of next 5 gameweeks with 'team_and_position' & average 'match_multiplier' 
next_3_gameweeks_multi_pivot = next_3_gameweeks_multi_df.pivot_table(index=['team_and_position'],
                                                          values=['match_multiplier'], 
                                                          aggfunc=np.mean)                                                  
# Transforming the pivot table back into a dataframe
next_3_gameweeks_multi_df = pd.DataFrame(next_3_gameweeks_multi_pivot.to_records()).rename(columns={"match_multiplier": "nxt_3_gw_avg_multi"})

print("--- 'next_3_gameweeks_multi_df' dataframe created ---")
# print(next_3_gameweeks_multi_df.head())

####################################

# Creating a dataframe of just the next gameweek with columns 'team_and_position' & 'match_multiplier'
gameweek_multiplier_df = slim_fix_df[slim_fix_df.is_current_gameweek == True].reset_index(drop = True)
gameweek_multiplier_df = gameweek_multiplier_df[['team_and_position', 'match_multiplier']]

# Renaming match multipiler to 'current_gw_multiplier'
gameweek_multiplier_df = gameweek_multiplier_df.rename(columns={"match_multiplier": "gw_multi"})

# Pulling the 5 week average match multiplier into 'gameweek_multiplier_df' 
gameweek_multiplier_df['nxt_5_gw_multi'] = gameweek_multiplier_df.team_and_position.map(next_5_gameweeks_multi_df.set_index('team_and_position').nxt_5_gw_avg_multi)
gameweek_multiplier_df['nxt_3_gw_multi'] = gameweek_multiplier_df.team_and_position.map(next_3_gameweeks_multi_df.set_index('team_and_position').nxt_3_gw_avg_multi)

# Taking an average of of the current gameweek multiplier and the next 5 week match multiplier
# This way match strength will still be a little weighted towards the next match
# However the upcoming matches are also taken into account to avoid have to transfer players out of the team again in upcoming weeks
gameweek_multiplier_df['nxt_3_wk_blended_match_multi'] = gameweek_multiplier_df[['gw_multi','nxt_3_gw_multi']].mean(axis=1)
    
# Gameweek 5 multiplier not being used currently
gameweek_multiplier_df['nxt_5_wk_blended_match_multi'] = gameweek_multiplier_df[['gw_multi','nxt_5_gw_multi']].mean(axis=1)

print("--- 'gameweek_multiplier_df' dataframe created ---")

### Player History Data ###
### Data frame containing a row per player history for each previous gameweek ###
## Primarily use to determine the appearances and average gameweek multiplier for games played
## these will be used to determine points per gameweek and then used to calculate a predicted player score ###

#######################################
### Create Player History Dataframe ###
#######################################

start_time = time.time()

## Creating player_history df using the function created earlier
player_history_df = create_player_history_table()

print("--- 'player_history_df' dataframe created ---")

time_taken(start_time = start_time)

# player_history_df.head(50)

####################################################################
###        Creating a slimline version of player history         ###
### and combining with gameweek multipliers and full player data ###
####################################################################

### Used to provide a score per match, adjusted based on that match difficulty

# Creating a slimline version of the player history dataframe
slim_player_history_df = player_history_df[['element', 'opponent_team', 'total_points', 'round', 'minutes']].copy()

# Enruching this seasons player history by adding previous season average as gameweek 0
slim_player_history_df = pd.concat([gameweek_zero_df, slim_player_history_df])

# mapping opponent team_abv, filling with 'PRE' if cannot be found as this will be the previous season rather than a specific team
slim_player_history_df['opponent_abv'] = slim_player_history_df.opponent_team.map((club_df.set_index('id').short_name)).fillna('PRE')

# Renaming the 'round' field to 'gameweek'
slim_player_history_df = slim_player_history_df.rename(columns={'round':'gameweek', 'total_points': 'match_points'})

# merging with full player df to pull through required fields
slim_player_history_df = slim_player_history_df.merge(full_player_df[['id',
                                                                      'first_name',
                                                                      'second_name',
                                                                      'web_name',
                                                                      'position_abv',
                                                                      'team_abv']], 
                                                       how = 'left', left_on = ['element'], right_on = ['id']).copy()

# Combining the team and position abbreviations to create a lookup field for the fixture dataframe
slim_player_history_df['team_and_position'] = slim_player_history_df.team_abv + '-' + slim_player_history_df.position_abv

slim_player_history_df = slim_player_history_df.merge(slim_fix_df[['gameweek', 'team_and_position', 'match_multiplier']], 
                                                      how = 'left', left_on = ['gameweek', 'team_and_position'], right_on = ['gameweek', 'team_and_position']).copy()

# filling null match multipliers with 1 
# (should only be the gameweek 0 records as there is not a relative gameweek to merge with in slim_fix_df)
slim_player_history_df['match_multiplier'] = slim_player_history_df['match_multiplier'].fillna(1)

# creating appearance field, 1 if played more than one minute, 0 if no mins played that gameweek
slim_player_history_df['appearance'] = slim_player_history_df.minutes.apply(lambda x: 1 if x > 0 else 0)

# Assigning an appearance factor based on the number of mins they played. 1 if over 60 mins, otherwise the mins played / 60, 
slim_player_history_df['appearance_factor'] = slim_player_history_df.minutes.apply(lambda x: 1.0 if x > 60 else (x / 90))

# Adjusting the match points based on match difficulty, so tougher matches have points increased and easier matches have points decreased
slim_player_history_df['adjusted_match_points'] = slim_player_history_df['match_points'] / (slim_player_history_df['match_multiplier'] / 1)
slim_player_history_df['adjusted_match_points'] = slim_player_history_df['adjusted_match_points'].astype(float)

## Removing records where they did not play
slim_player_history_df = slim_player_history_df[slim_player_history_df.appearance == 1]

slim_player_history_df = slim_player_history_df[['gameweek',
                                                 'id',
                                                 'first_name',
                                                 'second_name',
                                                 'web_name',
                                                 'team_abv',
                                                 'position_abv',
                                                 'team_and_position',
                                                 'opponent_abv',
                                                 'match_points',                                 
                                                 'match_multiplier',
                                                 'adjusted_match_points',
                                                 'minutes',
                                                 'appearance',
                                                 'appearance_factor']].copy()

print("--- 'slim_player_history_df' dataframe created ---")

# create average points per appearance pivot
avg_points_per_app_df = slim_player_history_df[['id','adjusted_match_points']].copy()
avg_points_per_app_df['adjusted_match_points'] = avg_points_per_app_df['adjusted_match_points'].astype(float)
avg_points_per_app_pivot = avg_points_per_app_df.pivot_table(index=['id'], values=['adjusted_match_points'],aggfunc='mean')
# Then converting to a dataframe and renaming 
avg_points_per_app_df = pd.DataFrame(avg_points_per_app_pivot.to_records()).rename(columns={"adjusted_match_points": "avg_points_per_app"})
print("--- 'avg_points_per_app_df' dataframe created ---")

# Dataframe showing appearances per player this season
season_appearances_df = slim_player_history_df[slim_player_history_df.gameweek != 0].copy()
season_appearances_pivot = season_appearances_df.pivot_table(index=['id'], values=['appearance'],aggfunc='sum')
season_appearances_df = pd.DataFrame(season_appearances_pivot.to_records()).rename(columns={"appearance": "season_appearances"})
print("--- 'season_appearances_df' dataframe created ---")

# create total points pivot including 'gameweek zero'
total_points_df = slim_player_history_df[['id','match_points']].copy()
total_points_df['match_points'] = total_points_df['match_points'].astype(float)
total_points_pivot = total_points_df.pivot_table(index=['id'], values=['match_points'],aggfunc='sum')
# Then converting to a dataframe and renaming 
total_points_df = pd.DataFrame(total_points_pivot.to_records()).rename(columns={"match_points": "total_points_inc_gw_0"})
print("--- 'total_points_df' dataframe created ---")

#############################################################
### Updating slim_player_df with the gameweek multipliers ###
#############################################################

# Creating a slimline version of the elements table with relevant data points required for players
slim_player_df = full_player_df[['first_name', 'second_name', 'web_name', 'id', 'position_abv',
                                 'team_abv', 'team_and_position', 'selected_by_percent', 'current_gameweek',
                                 'this_round_play_chance', 'nxt_round_play_chance', 'status',
                                 'cost_in_mil', 'minutes', 'net_transfers_this_gameweek','net_transfers_this_season', 'total_points',
                                 'total_points_inc_prv_season', 'prev_season_minutes', 'prev_season_points',
                                 'prev_season_points_per_gw', 'prev_season_points_per_mil', 'points_per_mil']].copy()

# Adding current game week multipliers from gameweek_multiplier_df
slim_player_df = slim_player_df.merge(gameweek_multiplier_df[['team_and_position',
                                                              'gw_multi',
                                                              'nxt_3_gw_multi',
                                                              'nxt_3_wk_blended_match_multi',
                                                              'nxt_5_gw_multi',
                                                              'nxt_5_wk_blended_match_multi']], 
                                                       how = 'left', on = ['team_and_position']).copy()

# Using the players average points per appearance calculate the predicted score for:
    # next gameweek using next gameweek multiplier
    # average for next 3 gameweeks using the average of the next 3 gameweeks
    # average for next 5 gameweeks using the average of the next 5 gameweeks
slim_player_df['avg_points_per_app'] = slim_player_df.id.map(avg_points_per_app_df.set_index('id').avg_points_per_app)
slim_player_df['nxt_gw_pred_score']  = (slim_player_df['avg_points_per_app'] * slim_player_df['gw_multi']) * slim_player_df['nxt_round_play_chance']
slim_player_df['nxt_3gw_pred_score'] = (slim_player_df['avg_points_per_app'] * slim_player_df['nxt_3_gw_multi']) * slim_player_df['nxt_round_play_chance']
slim_player_df['nxt_5gw_pred_score'] = (slim_player_df['avg_points_per_app'] * slim_player_df['nxt_5_gw_multi']) * slim_player_df['nxt_round_play_chance']
slim_player_df['nxt_gw_pred_score_per_mil'] =  slim_player_df['nxt_gw_pred_score']  / slim_player_df['cost_in_mil']
slim_player_df['nxt_3gw_pred_score_per_mil'] = slim_player_df['nxt_3gw_pred_score'] / slim_player_df['cost_in_mil']
slim_player_df['nxt_5gw_pred_score_per_mil'] = slim_player_df['nxt_5gw_pred_score'] / slim_player_df['cost_in_mil']

slim_player_df['club_games_played'] = slim_player_df.team_abv.map(club_games_played_df.set_index('team_abv').games_played)
slim_player_df['appearances_this_season'] = slim_player_df.id.map(season_appearances_df.set_index('id').season_appearances).fillna(0.0)
slim_player_df['appearance_percentage'] = slim_player_df.appearances_this_season / slim_player_df.club_games_played
slim_player_df['total_points_inc_gw_0'] = slim_player_df.id.map(total_points_df.set_index('id').total_points_inc_gw_0).fillna(0.0)

##################################
###### Player Score V2 Calc ######
##################################

# Player score v2 calc = 
        # Pedicted player score for the next 3 gameweeks as points per million:
            # Average points per appearance 
            # multiplied by average gw multiplier for next 3 weeks 
            # Multiplied by chance to play next round) 
            # Divided by cost (in million)

calc_player_score_v2 = slim_player_df.nxt_3gw_pred_score_per_mil * slim_player_df['nxt_3_wk_blended_match_multi']

slim_player_df['player_score_v2'] = calc_player_score_v2

########################################
### End of player score calculations ###
########################################

slim_player_df = slim_player_df.sort_values(by=['player_score_v2'], ascending=False).reset_index(drop = True)

print("--- 'slim_player_df' updated ---")
print("--- all operations complete ---")

##########################################################
### TOP POINTS PLAYERS AND TOP PLAYER SCORE DATAFRAMES ###
###           BASED ON PLAYER SCORE CALC V2            ###
##########################################################


######################################################################
# Creating a data frame of players ordered by their predicted points #
######################################################################

points_top_players = slim_player_df[['cost_in_mil',
                                     'team_abv',
                                     'position_abv',
                                     'web_name',
                                     'id',
                                     'total_points_inc_gw_0',
                                     'avg_points_per_app',
                                     'gw_multi',
                                     'nxt_gw_pred_score',
                                     'nxt_3_gw_multi',
                                     'nxt_3gw_pred_score',
                                     'player_score_v2',
                                     'appearance_percentage',
                                     'net_transfers_this_gameweek',
                                     'net_transfers_this_season'
                                    ]].copy()

points_top_players = points_top_players[points_top_players['player_score_v2'] > 0.0]
points_top_players = points_top_players.sort_values(by=['nxt_3gw_pred_score'], ascending=False).reset_index(drop = True)

print("--- 'points_top_players' dataframe created ---")

###########################################################################################################
# Creating a data frame of players ordered by their player score (to get the most cost effective players) #
###########################################################################################################

player_score_top_players = slim_player_df[['cost_in_mil',
                                           'team_abv',
                                           'position_abv',
                                           'web_name',
                                           'id',
                                           'total_points_inc_gw_0',
                                           'avg_points_per_app',
                                           'gw_multi',
                                           'nxt_gw_pred_score',
                                           'nxt_3_gw_multi',
                                           'nxt_3gw_pred_score',
                                           'player_score_v2',
                                           'appearance_percentage',
                                           'net_transfers_this_gameweek',
                                           'net_transfers_this_season'
                                        ]].copy()

player_score_top_players = player_score_top_players[player_score_top_players['player_score_v2'] > 0.0]
player_score_top_players = player_score_top_players.sort_values(by=['player_score_v2'], ascending=False).reset_index(drop = True)

print("--- 'player_score_top_players' dataframe created ---")
print("--- all operations complete ---")


--- 'club_df' dataframe created ---
--- 'prev_season_player_df' dataframe created ---
--- 'prev_season_player_history_df' dataframe created ---
--- 'gameweek_zero_df' dataframe created ---
--- 'full_player_df' dataframe created ---
--- 'fix_df' dataframe created ---
--- 'slim_fix_df' dataframe created ---
--- 'next_5_gameweeks_multi_df' dataframe created ---
--- 'next_3_gameweeks_multi_df' dataframe created ---
--- 'gameweek_multiplier_df' dataframe created ---
--- all operations complete ---
--- Starting to create player history dataframe ---
--- 'player_history_df' dataframe created ---
--- Time taken to run: 1 minutes 20 seconds  ---
--- 'slim_player_history_df' dataframe created ---
--- 'avg_points_per_app_df' dataframe created ---
--- 'season_appearances_df' dataframe created ---
--- 'total_points_df' dataframe created ---
--- 'slim_player_df' updated ---
--- all operations complete ---
--- 'points_top_players' dataframe created ---
--- 'player_score_top_players' dataframe created

## 3. Creating Optimal Team Function

In [4]:
#########################################################
### Creating optimal team using optimal team function ###
#########################################################

################################################
### Optimal team logic Using Player Score V2 ###
################################################

# Function which produces the optimal team based on:
    # A specified number of star players (picking solely based on top points)
    # Filling the rest of the team based on points per million 
    # A set the budget you want to spend on your team
    # Specify the minimum cost you want left to spend on the final players (most likely your subs)

def get_optimal_team_objects(points_top_players=points_top_players, 
                             player_score_top_players=player_score_top_players,
                             budget = 100, 
                             star_player_limit = 3, 
                             gk = 2, 
                             df = 5, 
                             md = 5, 
                             fwd = 3, 
                             min_player_cost = 4.5,
                             min_appearance_percentage = 0.8,
                             min_gw_multi = 1):
    
    optimal_team = pd.DataFrame(columns = ['id', 
                                           'web_name', 
                                           'position_abv',
                                           'team_abv',
                                           'cost_in_mil',
                                           'total_points_inc_gw_0',
                                           'gw_multi',
                                           'nxt_gw_pred_score',
                                           'nxt_3_gw_multi', 
                                           'nxt_3gw_pred_score', 
                                           'appearance_percentage',
                                           'player_score_v2',
                                           'net_transfers_this_gameweek',
                                           'net_transfers_this_season'])
    star_player_limit = star_player_limit
    budget = budget
    min_gw_multi = min_gw_multi
    positions = {'GKP': gk, 'DEF': df, 'MID': md, 'FWD': fwd}
    club_player_count = dict((club, 3) for club in list(club_df['short_name'].unique()))
    players_left = 15
    min_player_cost = min_player_cost
    min_appearance_percentage = min_appearance_percentage
    
    for index, player in points_top_players.iterrows():
        if      len(optimal_team) < star_player_limit \
                and budget >= player.cost_in_mil \
                and positions[player.position_abv] > 0 \
                and club_player_count[player.team_abv] > 0 \
                and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost) \
                and player.appearance_percentage >= min_appearance_percentage \
                and player.nxt_3_gw_multi >= min_gw_multi \
                and player.gw_multi >= min_gw_multi:
            my_dict = {'id': player.id,
                       'web_name': player.web_name, 
                       'position_abv': player.position_abv, 
                       'cost_in_mil': player.cost_in_mil,
                       'total_points_inc_gw_0': player.total_points_inc_gw_0,
                       'gw_multi': player.gw_multi,
                       'nxt_gw_pred_score': player.nxt_gw_pred_score,
                       'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                       'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                       'team_abv': player.team_abv,
                       'appearance_percentage': player.appearance_percentage,
                       'player_score_v2': player.player_score_v2,
                       'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                       'net_transfers_this_season': player.net_transfers_this_season}
            optimal_team = optimal_team.append(my_dict, ignore_index=True)
            budget -= player.cost_in_mil
            players_left = players_left - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            club_player_count[player.team_abv] = club_player_count[player.team_abv] -1
    for index, player in player_score_top_players.iterrows():
        if      player.id not in optimal_team['id'].to_list() \
                and budget >= player.cost_in_mil \
                and positions[player.position_abv] > 0 \
                and club_player_count[player.team_abv] > 0 \
                and (budget - player.cost_in_mil) >= ((players_left - 1) * min_player_cost) \
                and player.appearance_percentage >= min_appearance_percentage \
                and player.nxt_3_gw_multi >= min_gw_multi \
                and player.gw_multi >= min_gw_multi:
            my_dict2 = {'id':player.id,
                        'web_name': player.web_name, 
                        'position_abv': player.position_abv, 
                        'cost_in_mil': player.cost_in_mil, 
                        'total_points_inc_gw_0': player.total_points_inc_gw_0,
                        'gw_multi': player.gw_multi,
                        'nxt_gw_pred_score': player.nxt_gw_pred_score,
                        'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                        'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                        'team_abv': player.team_abv,
                        'appearance_percentage': player.appearance_percentage,
                        'player_score_v2': player.player_score_v2,
                        'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                        'net_transfers_this_season': player.net_transfers_this_season}
            optimal_team = optimal_team.append(my_dict2, ignore_index=True) 
            budget -= player.cost_in_mil
            players_left = players_left - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            club_player_count[player.team_abv] = club_player_count[player.team_abv] -1
    return optimal_team

print("--- 'get_optimal_team_objects' function created ---")

# functions to evaluate optimal team 
def valid_team(team):
    if (len(team)) < 15:
        return 'Valid Team: False - Cannot make selection of 15 players with current star_player_limit, budget & min_player_cost'
    else:
        return 'Valid Team: True'
    
print("--- 'valid_team' function created ---")
    
def terrible_subs(team):
    if team.nxt_3gw_pred_score.min() < 3:
        return 'Low Scoring Subs! Some players have less than 3 points on average over next 3 gameweeks'
    else:
        return 'Great Subs! All players expected to score more than 3 points on average over next 3 gameweeks'
    
print("--- 'terrible_subs' function created ---")

### Create database of optimal teams
### Using all possible combinations of a range of start player limit, budget and min player cost 
## Then pivot that table to find highest total player score for the top 11 where subs are min 3 points

import itertools

start_time = time.time()

def create_optimal_team_combos(budget_list, 
                               star_player_limit_list, 
                               min_player_cost_list, 
                               min_appearance_percentage = 0.75,
                               min_gw_multi = 1):
    all_optimal_teams_df = pd.DataFrame(columns = ['id', 
                                                   'web_name', 
                                                   'position_abv',
                                                   'team_abv',
                                                   'cost_in_mil', 
                                                   'gw_multi',
                                                   'nxt_gw_pred_score',
                                                   'nxt_3_gw_multi', 
                                                   'nxt_3gw_pred_score', 
                                                   'player_score_v2',
                                                   'budget_limit',
                                                   'star_player_limit',
                                                   'min_player_cost'
                                                  ])
    budget_list = budget_list
    star_player_limit_list = star_player_limit_list
    min_player_cost_list = min_player_cost_list
    min_appearance_percentage = min_appearance_percentage
    min_gw_multi = min_gw_multi
    print("--- Running 'create_optimal_team_combos' function with the following inputs ---")
    print("    - Budgets                   = " + str(budget_list))
    print("    - Star Player Limits        = " + str(star_player_limit_list))
    print("    - Min Player Costs          = " + str(min_player_cost_list))
    print("    - Min Appearance Percentage = " + str(int(min_appearance_percentage * 100)) + '%')
    print("    - Min Gameweek Multiplier   = " + str(min_gw_multi))
    
    for x, y, z in itertools.product(budget_list, star_player_limit_list, min_player_cost_list):
        combo_list = [x,y,z]
        optimal_team_df = get_optimal_team_objects(budget = combo_list[0], 
                                                   star_player_limit = combo_list[1], 
                                                   min_player_cost = combo_list[2],
                                                   min_appearance_percentage = min_appearance_percentage,
                                                   min_gw_multi = min_gw_multi)
        optimal_team_df['budget_limit'] = combo_list[0]
        optimal_team_df['star_player_limit'] = combo_list[1]
        optimal_team_df['min_player_cost'] = combo_list[2]
        all_optimal_teams_df = pd.concat([all_optimal_teams_df, optimal_team_df])
    all_optimal_teams_df['combo_str'] = all_optimal_teams_df['budget_limit'].astype(str) + '-' + all_optimal_teams_df['star_player_limit'].astype(str) + '-' + all_optimal_teams_df['min_player_cost'].astype(str)
    all_optimal_teams_df['combo_list'] = all_optimal_teams_df[['budget_limit','star_player_limit','min_player_cost']].values.tolist()
    return all_optimal_teams_df  

print("--- 'create_optimal_team_combos' function created ---") 

# entering the range for each input in  
budget_list = [99, 98, 97, 96]
star_player_limit_list = [1, 2, 3, 4, 5, 6, 7, 8]
min_player_cost_list = [4.5, 5, 5.5, 6]
min_gw_multi = .98
min_appearance_percentage = 0.75 
                                                   
all_optimal_teams_df = create_optimal_team_combos(budget_list, 
                                                  star_player_limit_list, 
                                                  min_player_cost_list,
                                                  min_gw_multi,
                                                  min_appearance_percentage
                                                 )

print('')        
print("--- 'all_optimal_teams_df' dataframe created ---")  
   
# create a pivot table showing the total predicted points per combination
all_optimal_teams_pivot = all_optimal_teams_df.pivot_table(index=['combo_str'],
                                                      values=['nxt_3gw_pred_score'], 
                                                      aggfunc='sum')

total_predicted_points_per_combo = pd.DataFrame(all_optimal_teams_pivot.to_records()).rename(columns={"nxt_3gw_pred_score": "total_pred_score"})

# Convert to dataframe
total_predicted_points_per_combo = total_predicted_points_per_combo.sort_values(by=['total_pred_score'], ascending=False).reset_index(drop = True)

print('')
time_taken(start_time = start_time)
print("--- all operations complete ---")
total_predicted_points_per_combo.head(10)


--- 'get_optimal_team_objects' function created ---
--- 'valid_team' function created ---
--- 'terrible_subs' function created ---
--- 'create_optimal_team_combos' function created ---
--- Running 'create_optimal_team_combos' function with the following inputs ---
    - Budgets                   = [99, 98, 97, 96]
    - Star Player Limits        = [1, 2, 3, 4, 5, 6, 7, 8]
    - Min Player Costs          = [4.5, 5, 5.5, 6]
    - Min Appearance Percentage = 98%
    - Min Gameweek Multiplier   = 0.75

--- 'all_optimal_teams_df' dataframe created ---

--- Time taken to run: 0 minutes 18 seconds  ---
--- all operations complete ---


,combo_str,total_pred_score
0,97-3-4.5,48.544888
1,99-4-5.5,48.544888
2,99-4-4.5,48.544888
3,99-3-6.0,48.544888
4,99-3-5.5,48.544888
5,99-3-5.0,48.544888
6,99-3-4.5,48.544888
7,98-5-4.5,48.544888
8,97-3-5.0,48.544888
9,97-3-5.5,48.544888


## 4. Creating Optimal Team

In [5]:
budget = 76
star_player_limit = 3
min_player_cost = 4.5
min_gw_multi = .98
min_appearance_percentage = 0.75 
# Setting the minimum percentage of appearances for their club, might need a workaround for people on loan or transferred in mid season
    

# Run optimal team function to create optimal_team_dataframe with custom parameters    
optimal_team_df = get_optimal_team_objects(budget = budget,
                                           star_player_limit = star_player_limit,                                         
                                           min_player_cost = min_player_cost,
                                           min_appearance_percentage = min_appearance_percentage,
                                           min_gw_multi = min_gw_multi)  


print('Optimal Team For Gameweek: ' + str(current_gameweek))
print('--------------------------')
print('Total Team Points: ' + str(round(optimal_team_df['nxt_3gw_pred_score'].sum(),2)))
print('Total Team Cost: ' + str(round(optimal_team_df['cost_in_mil'].sum(),2)))
print('Total Team Multiplier: ' + str(round(optimal_team_df['nxt_3_gw_multi'].sum(),2)))
print(valid_team(optimal_team_df))
print('--------------------------')
print(terrible_subs(optimal_team_df))

print("--- all operations complete ---")
# export_to_csv(dataframe = optimal_team_df, filename = 'Optimal_Team_gw_'+str(current_gameweek))
optimal_team_df.head(15)

Optimal Team For Gameweek: 12
--------------------------
Total Team Points: 36.36
Total Team Cost: 48.7
Total Team Multiplier: 9.29
Valid Team: False - Cannot make selection of 15 players with current star_player_limit, budget & min_player_cost
--------------------------
Low Scoring Subs! Some players have less than 3 points on average over next 3 gameweeks
--- all operations complete ---


,id,web_name,position_abv,team_abv,cost_in_mil,total_points_inc_gw_0,gw_multi,nxt_gw_pred_score,nxt_3_gw_multi,nxt_3gw_pred_score,appearance_percentage,player_score_v2,net_transfers_this_gameweek,net_transfers_this_season
0,254,Salah,MID,LIV,12.2,79.852941,1.326733,9.370093,1.187851,8.389239,0.818182,0.864567,614,-252245
1,431,Fabianski,GKP,WHU,5.1,56.320000,1.082949,5.418182,1.019978,5.103125,0.909091,1.052108,313,163547
2,213,Meslier,GKP,LEE,4.6,43.000000,0.991342,4.309524,0.985856,4.285674,0.909091,0.921046,155,220007
3,442,Masuaku,DEF,WHU,4.5,37.529412,1.082949,4.011710,1.019978,3.778437,0.818182,0.882864,19,132820
4,200,Dallas,DEF,LEE,4.5,38.000000,0.991342,3.856710,0.985856,3.835367,0.909091,0.842586,268,168181
5,42,Targett,DEF,AVL,4.5,32.857143,0.995833,3.157440,1.067345,3.384178,1.000000,0.775796,2,114314
6,491,Koch,DEF,LEE,4.5,31.000000,0.991342,3.119048,0.985856,3.101786,0.909091,0.681427,4,10933
7,518,Mendy,MID,LEI,4.5,25.571429,1.128440,2.507745,1.040812,2.313006,0.909091,0.557499,-40,141890
8,513,Lewis,DEF,NEW,4.3,21.714286,1.014706,2.220525,0.992793,2.172572,1.000000,0.507144,112,21435


## 5. Creating My Team and Comparing to Optimal Team

In [6]:
### MY TEAM PICKS DATA ###
### Creating a dataframe for my team picks data and saving as csv ###
### Used to see value of current team and highlight:
            # Players to use as captain / subs   
            # When to use wildcards
            # Weak players to transfer out
                    
#############################################
#### my_team_picks_df dataframe creation ####
#############################################

# ## Getting my team data from the api url, using session from earlier as login details required
my_team_ses = (session.get(team_url))

## Converting that data to json then dataframe
my_team_json = my_team_ses.json()
my_team_picks_df = pd.DataFrame(my_team_json['picks'])

my_team_picks_df['potential_profit'] = my_team_picks_df.selling_price - my_team_picks_df.purchase_price
my_team_picks_df['cost_in_mil'] = my_team_picks_df.selling_price / 10  

# Pulling through player data from 'full_player_df'
my_team_picks_df = my_team_picks_df.merge(slim_player_df[['id',
                                                          'first_name',
                                                          'second_name',
                                                          'web_name',
                                                          'total_points',
                                                          'points_per_mil',
                                                          'position_abv',
                                                          'team_abv',
                                                          'prev_season_points_per_gw',
                                                          'nxt_round_play_chance',
                                                          'nxt_gw_pred_score',
                                                          'nxt_3gw_pred_score',
                                                          'gw_multi',
                                                          'nxt_5_gw_multi',
                                                          'nxt_5_wk_blended_match_multi' ,
                                                          'nxt_3_gw_multi',
                                                          'nxt_3_wk_blended_match_multi' ,
                                                          'player_score_v2',
                                                          'net_transfers_this_gameweek',
                                                          'net_transfers_this_season']], 
                                           how = 'left', left_on = ['element'], right_on = ['id'])

my_team_picks_df['team_and_position'] = my_team_picks_df['team_abv'].astype(str) + '-' + my_team_picks_df['position_abv'].astype(str)
my_team_picks_df = my_team_picks_df.sort_values(by=['nxt_gw_pred_score'], ascending=False).reset_index(drop = True)
    
print("--- 'my_team_picks_df' dataframe created ---") 

# Creating a slim version of my_team which has the same fields as optimal_team_df
my_team_df = my_team_picks_df[['id',
                               'web_name',
                               'position_abv',
                               'cost_in_mil',
                               'gw_multi',
                               'nxt_gw_pred_score',
                               'nxt_3_gw_multi',
                               'nxt_3gw_pred_score',
                               'team_abv',
                               'player_score_v2',
                               'net_transfers_this_gameweek',
                               'net_transfers_this_season'
                              ]].copy()

# Creating a function to highlight the captain / vice captain choices
def captain_vice_capatin_selection(team_dataframe):
    my_team_df = team_dataframe.copy()
    my_capt_list = []
    my_team_df = my_team_df[my_team_df.position_abv != 'GKP']
    # create columns for score rank, multiplier rank, then average of these two, then rank again
    my_team_df['score_rank'] = my_team_df['nxt_gw_pred_score'].rank(ascending=False)
    my_team_df['multi_rank'] = my_team_df['gw_multi'].rank(ascending=False)
    my_team_df['avg_rank'] = my_team_df[['score_rank', 'multi_rank']].mean(axis=1)
    my_team_df['captain_rank'] = my_team_df['avg_rank'].rank()
    # sort by this rank again, using higest score as the decider if the ranks are the same
    my_team_df = my_team_df.sort_values(by=['captain_rank', 'score_rank'])
    # for loop to pick the first two rows from the reordered datframe and assign as captain and vice captain
    for player in my_team_df.iterrows():
        if len(my_capt_list) < 2:
            my_new_list = [player[1].web_name]
            my_capt_list = my_capt_list + my_new_list
    return (my_capt_list,'Captain: ' + my_capt_list[0] + ' / Vice Captain: ' + my_capt_list[1])   

print("--- 'captain_vice_capatin_selection' function created ---")

# Creating funtion to pickout the top 11 and subs from a team dataframe passed through the function
def create_first_11_and_subs(team_dataframe):
    team_dataframe = team_dataframe
    # Sorting dataframe by nxt_gw_pred_score
    team_dataframe = team_dataframe.sort_values(by=['nxt_gw_pred_score'], ascending=False).reset_index(drop = True)
    GKP_team_dataframe = team_dataframe[team_dataframe.position_abv == 'GKP'].copy()
    my_team_first_11 = pd.DataFrame(columns = ['id', 
                                               'web_name', 
                                               'position_abv',
                                               'team_abv',
                                               'cost_in_mil', 
                                               'gw_multi',
                                               'nxt_gw_pred_score',
                                               'nxt_3_gw_multi', 
                                               'nxt_3gw_pred_score', 
                                               'player_score_v2',
                                               'net_transfers_this_gameweek',
                                               'net_transfers_this_season'])
    my_team_subs = pd.DataFrame(columns = ['id', 
                                           'web_name', 
                                           'position_abv',
                                           'team_abv',
                                           'cost_in_mil', 
                                           'gw_multi',
                                           'nxt_gw_pred_score',
                                           'nxt_3_gw_multi', 
                                           'nxt_3gw_pred_score', 
                                           'player_score_v2',
                                           'net_transfers_this_gameweek',
                                           'net_transfers_this_season'])
    first_11_player_limit = 11
    positions = {'GKP': 1, 'DEF': 5, 'MID': 5, 'FWD': 3}
    for index, player in GKP_team_dataframe.iterrows():
        if first_11_player_limit > 0 and positions[player.position_abv] > 0:
            my_dict = {'id': player.id,
                       'web_name': player.web_name, 
                       'position_abv': player.position_abv, 
                       'cost_in_mil': player.cost_in_mil,
                       'gw_multi': player.gw_multi,
                       'nxt_gw_pred_score': player.nxt_gw_pred_score,
                       'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                       'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                       'team_abv': player.team_abv,
                       'player_score_v2': player.player_score_v2,
                       'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                       'net_transfers_this_season': player.net_transfers_this_season}
            my_team_first_11 = my_team_first_11.append(my_dict, ignore_index=True)
            first_11_player_limit = first_11_player_limit - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
    
    
    for index, player in team_dataframe.iterrows():
        if first_11_player_limit > 0 and positions[player.position_abv] > 0:
            my_dict = {'id': player.id,
                       'web_name': player.web_name, 
                       'position_abv': player.position_abv, 
                       'cost_in_mil': player.cost_in_mil,
                       'gw_multi': player.gw_multi,
                       'nxt_gw_pred_score': player.nxt_gw_pred_score,
                       'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                       'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                       'team_abv': player.team_abv,
                       'player_score_v2': player.player_score_v2,
                       'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                       'net_transfers_this_season': player.net_transfers_this_season}
            my_team_first_11 = my_team_first_11.append(my_dict, ignore_index=True)
            first_11_player_limit = first_11_player_limit - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            
    for index, player in team_dataframe.iterrows():
        if player.id not in my_team_first_11['id'].to_list():
            my_dict2 = {'id':player.id,
                        'web_name': player.web_name, 
                        'position_abv': player.position_abv, 
                        'cost_in_mil': player.cost_in_mil, 
                        'gw_multi': player.gw_multi,
                        'nxt_gw_pred_score': player.nxt_gw_pred_score,
                        'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                        'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                        'team_abv': player.team_abv,
                        'player_score_v2': player.player_score_v2,
                        'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                        'net_transfers_this_season': player.net_transfers_this_season}
            my_team_subs = my_team_subs.append(my_dict2, ignore_index=True) 
    return (my_team_first_11, my_team_subs)

print("--- 'create_first_11_and_subs' function created ---")

def create_first_11_and_subs_for_3_gw_average(team_dataframe):
    # function variables
    team_dataframe = team_dataframe
    # sorting dataframe by nxt_3gw_pred_score
    team_dataframe = team_dataframe.sort_values(by=['nxt_3gw_pred_score'], ascending=False).reset_index(drop = True)
    GKP_team_dataframe = team_dataframe[team_dataframe.position_abv == 'GKP'].copy()
    my_team_first_11 = pd.DataFrame(columns = ['id', 
                                               'web_name', 
                                               'position_abv',
                                               'team_abv',
                                               'cost_in_mil', 
                                               'gw_multi',
                                               'nxt_gw_pred_score',
                                               'nxt_3_gw_multi', 
                                               'nxt_3gw_pred_score', 
                                               'player_score_v2',
                                               'net_transfers_this_gameweek',
                                               'net_transfers_this_season'])
    my_team_subs = pd.DataFrame(columns = ['id', 
                                           'web_name', 
                                           'position_abv',
                                           'team_abv',
                                           'cost_in_mil', 
                                           'gw_multi',
                                           'nxt_gw_pred_score',
                                           'nxt_3_gw_multi', 
                                           'nxt_3gw_pred_score', 
                                           'player_score_v2',
                                           'net_transfers_this_gameweek',
                                           'net_transfers_this_season'])
    first_11_player_limit = 11
    gk_limit = 1
    positions = {'GKP': 1, 'DEF': 5, 'MID': 5, 'FWD': 3}
    # Iteration:
    
    # Picking first 11 GKP first
    for index, player in GKP_team_dataframe.iterrows():
        if first_11_player_limit > 0 and positions[player.position_abv] > 0:
            my_dict = {'id': player.id,
                       'web_name': player.web_name, 
                       'position_abv': player.position_abv, 
                       'cost_in_mil': player.cost_in_mil,
                       'gw_multi': player.gw_multi,
                       'nxt_gw_pred_score': player.nxt_gw_pred_score,
                       'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                       'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                       'team_abv': player.team_abv,
                       'player_score_v2': player.player_score_v2,
                       'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                       'net_transfers_this_season': player.net_transfers_this_season}
            my_team_first_11 = my_team_first_11.append(my_dict, ignore_index=True)
            first_11_player_limit = first_11_player_limit - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
            
    # Then pick the rest of the top 11
    for index, player in team_dataframe.iterrows():
        if first_11_player_limit > 0 and positions[player.position_abv] > 0:
            my_dict = {'id': player.id,
                       'web_name': player.web_name, 
                       'position_abv': player.position_abv, 
                       'cost_in_mil': player.cost_in_mil,
                       'gw_multi': player.gw_multi,
                       'nxt_gw_pred_score': player.nxt_gw_pred_score,
                       'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                       'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                       'team_abv': player.team_abv,
                       'player_score_v2': player.player_score_v2,
                       'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                       'net_transfers_this_season': player.net_transfers_this_season}
            my_team_first_11 = my_team_first_11.append(my_dict, ignore_index=True)
            first_11_player_limit = first_11_player_limit - 1 
            positions[player.position_abv] = positions[player.position_abv] -1
    # Then assign leftover players to subs df        
    for index, player in team_dataframe.iterrows():
        if player.id not in my_team_first_11['id'].to_list():
            my_dict2 = {'id':player.id,
                        'web_name': player.web_name, 
                        'position_abv': player.position_abv, 
                        'cost_in_mil': player.cost_in_mil, 
                        'gw_multi': player.gw_multi,
                        'nxt_gw_pred_score': player.nxt_gw_pred_score,
                        'nxt_3_gw_multi': player.nxt_3_gw_multi, 
                        'nxt_3gw_pred_score': player.nxt_3gw_pred_score, 
                        'team_abv': player.team_abv,
                        'player_score_v2': player.player_score_v2,
                        'net_transfers_this_gameweek': player.net_transfers_this_gameweek,
                        'net_transfers_this_season': player.net_transfers_this_season}
            my_team_subs = my_team_subs.append(my_dict2, ignore_index=True) 
    return (my_team_first_11, my_team_subs)

print("--- 'create_first_11_and_subs_for_3_gw_average' function created ---")

# list of the player ids of those in my team
my_team_id_list = list(my_team_df.id)

#########################################
### Comparing my team to optimal team ###
#########################################

# adding a column to optimal_team_df to see if they are
optimal_team_df['player_in_my_team'] = optimal_team_df['id'].apply(lambda x: 1 if x in my_team_id_list else 0)

my_team_first_11_df, my_team_subs_df = create_first_11_and_subs(my_team_df)
optimal_team_first_11_df, optimal_team_subs_df = create_first_11_and_subs(optimal_team_df)

#######################################################################
print('My Team Stats')
print('-------------')
print('Budget Available:')
print('  - £' + str(round(sum(my_team_df.cost_in_mil)+money_in_bank,1)) + 'm')
print('  - Team Value: £' + str(round(sum(my_team_df.cost_in_mil),1)) + 'm')
print('  - Cash In The Bank: £' + str(money_in_bank) + 'm')
print('Next Gameweek:')
print('  - First 11 Total Points:       ' + str(round(sum(my_team_first_11_df.nxt_gw_pred_score),2)))
print('  - First 11 Average GW Multi:   ' + str(round(np.mean(my_team_first_11_df.gw_multi),2)))
print('  - Subs Total Points:           ' + str(round(sum(my_team_subs_df.nxt_gw_pred_score),2)))
print('  - Subs Average GW Multi:       ' + str(round(np.mean(my_team_subs_df.gw_multi),2)))

print('Next 3 Gameweeks:')
print('  - First 11 Total Points:       ' + str(round(sum(my_team_first_11_df.nxt_3gw_pred_score),2)))
print('  - First 11 Average 3 GW Multi: ' + str(round(np.mean(my_team_first_11_df.nxt_3_gw_multi),2)))
print('  - Subs Total Points:           ' + str(round(sum(my_team_subs_df.nxt_3gw_pred_score),2)))
print('  - Subs Average 3 GW Multi:     ' + str(round(np.mean(my_team_subs_df.nxt_3_gw_multi),2)))

print('Captaincy Suggestion: ')
print('  - ' + captain_vice_capatin_selection(team_dataframe = my_team_first_11_df)[1])

print('Subs:')
print('  - ' + str(list(my_team_subs_df.web_name)))

#######################################################################

print('')
print('')

#######################################################################

print('Optimal Team Stats')
print('------------------')
print('Team Cost: £')
print('  - £' + str(round(sum(optimal_team_df.cost_in_mil),1)) + 'm')
print('Next Gameweek:')
print('  - First 11 Total Points:       ' + str(round(sum(optimal_team_first_11_df.nxt_gw_pred_score),2)))
print('  - First 11 Average GW Multi:   ' + str(round(np.mean(optimal_team_first_11_df.gw_multi),2)))
print('  - Subs Total Points:           ' + str(round(sum(optimal_team_subs_df.nxt_gw_pred_score),2)))
print('  - Subs Average GW Multi:       ' + str(round(np.mean(optimal_team_subs_df.gw_multi),2)))

print('Next 3 Gameweeks:')
print('  - First 11 Total Points:       ' + str(round(sum(optimal_team_first_11_df.nxt_3gw_pred_score),2)))
print('  - First 11 Average 3 GW Multi: ' + str(round(np.mean(optimal_team_first_11_df.nxt_3_gw_multi),2)))
print('  - Subs Total Points:           ' + str(round(sum(optimal_team_subs_df.nxt_3gw_pred_score),2)))
print('  - Subs Average 3 GW Multi:     ' + str(round(np.mean(optimal_team_subs_df.nxt_3_gw_multi),2)))

#######################################################################

## identifying players dropping in value
def player_value_dropping(dataframe):
    dataframe = dataframe[dataframe.net_transfers_this_gameweek < -50000]
    return dataframe

player_value_dropping_df = player_value_dropping(my_team_df)
player_value_dropping_df.head()

--- 'my_team_picks_df' dataframe created ---
--- 'captain_vice_capatin_selection' function created ---
--- 'create_first_11_and_subs' function created ---
--- 'create_first_11_and_subs_for_3_gw_average' function created ---
My Team Stats
-------------
Budget Available:
  - £99.8m
  - Team Value: £99.3m
  - Cash In The Bank: £0.5m
Next Gameweek:
  - First 11 Total Points:       70.72
  - First 11 Average GW Multi:   1.08
  - Subs Total Points:           17.51
  - Subs Average GW Multi:       1.05
Next 3 Gameweeks:
  - First 11 Total Points:       69.69
  - First 11 Average 3 GW Multi: 1.06
  - Subs Total Points:           16.29
  - Subs Average 3 GW Multi:     0.97
Captaincy Suggestion: 
  - Captain: Son / Vice Captain: Jota
Subs:
  - ['Vestergaard', 'Balbuena', 'McCarthy', 'Armstrong']


Optimal Team Stats
------------------
Team Cost: £
  - £48.7m
Next Gameweek:
  - First 11 Total Points:       33.66
  - First 11 Average GW Multi:   1.08
  - Subs Total Points:           4.31
  - Subs 

,id,web_name,position_abv,cost_in_mil,gw_multi,nxt_gw_pred_score,nxt_3_gw_multi,nxt_3gw_pred_score,team_abv,player_score_v2,net_transfers_this_gameweek,net_transfers_this_season


## 6. Viewing My First 11 / Subs and Optimal First 11 / Subs and Players

In [7]:
print('My First 11:')
my_team_first_11_df

My First 11:


,id,web_name,position_abv,team_abv,cost_in_mil,gw_multi,nxt_gw_pred_score,nxt_3_gw_multi,nxt_3gw_pred_score,player_score_v2,net_transfers_this_gameweek,net_transfers_this_season
0,431,Fabianski,GKP,WHU,5.1,1.082949,5.418182,1.019978,5.103125,1.052108,313,163547
1,390,Son,MID,TOT,9.2,1.128319,8.842920,1.038799,8.141331,0.928591,228,2020204
2,506,Wilson,FWD,NEW,6.5,1.104478,7.922772,1.054048,7.561027,1.255436,1164,227594
3,224,Vardy,FWD,LEI,10.1,1.086758,7.679452,0.989874,6.994833,0.705131,359,701012
4,302,Fernandes,MID,MUN,10.9,0.921348,7.110808,1.052709,8.124629,0.735710,636,1729950
5,468,Jota,MID,LIV,6.9,1.326733,6.314437,1.187851,5.653447,1.030150,1325,1925416
6,110,Zouma,DEF,CHE,5.3,1.112069,6.157020,1.101317,6.097493,1.205010,485,1384467
7,275,Mahrez,MID,MCI,8.3,1.085366,5.909507,1.180037,6.424966,0.866377,444,516954
8,164,Calvert-Lewin,FWD,EVE,7.6,0.899225,5.466731,0.948242,5.764726,0.665634,323,3656314
9,123,James,DEF,CHE,5.2,1.112069,5.017134,1.101317,4.968627,1.057451,464,612761


In [8]:
print('Set the following as subs:')
my_team_subs_df

Set the following as subs:


,id,web_name,position_abv,team_abv,cost_in_mil,gw_multi,nxt_gw_pred_score,nxt_3_gw_multi,nxt_3gw_pred_score,player_score_v2,net_transfers_this_gameweek,net_transfers_this_season
0,369,Vestergaard,DEF,SOU,4.7,1.013393,4.674695,0.949006,4.377686,0.894872,107,731593
1,443,Balbuena,DEF,WHU,4.4,1.082949,4.439294,1.019978,4.181157,0.999167,52,110243
2,363,McCarthy,GKP,SOU,4.6,1.013393,4.200951,0.949006,3.934041,0.839148,175,79583
3,368,Armstrong,MID,SOU,5.5,1.073733,4.197713,0.972408,3.801590,0.707144,-2,23857


## 7. Transfer Suggestions

In [28]:
# Function to suggest a transfer for a player

def transfer_suggestion(df_to_select_from, 
                        replacement_position, 
                        replacement_budget, 
                        player_team, 
                        min_appearance_percent = 0.8,
                        min_next_gw_multiplier = 1,
                        min_next_3_gw_multiplier = 1):
    # sets the player dataframe to that of the parameter, could be df of top points, could be df of top player score
    player_df = df_to_select_from.copy()
    
    # Creating a list of teams which have hit the player limit
    club_limit_df = my_team_df.groupby(['team_abv']).size().reset_index(name='counts')
    club_limit_df = club_limit_df[club_limit_df.counts >= 3] 
    club_limit_list = list(club_limit_df.team_abv)    
    if player_team in club_limit_list:
        club_limit_list = club_limit_list.remove('player_team')
    else:
        club_limit_list = club_limit_list
    
    # removing players where we have exceeded the player limits for that team using the above list
    player_df['club_limit_exceeded'] = player_df['team_abv'].apply(lambda x: True if x in club_limit_list else False)   
    player_df = player_df[player_df.club_limit_exceeded == False]
    
    # removing players whose appearance percentsage is less than the specified min accepted appearance percentage
    player_df = player_df[player_df.appearance_percentage >= min_appearance_percent]
    
    # removing players whose next gameweek multiplier is less than the specified min_next_gw_multiplier
    player_df = player_df[player_df.gw_multi >= min_next_gw_multiplier]

    # removing players whose next 3 gameweek multiplier is less than the specified min_next_3_gw_multiplier
    player_df = player_df[player_df.nxt_3_gw_multi >= min_next_3_gw_multiplier]
    
    # adds an 'in my team column' then removes any players already in the team
    player_df['in_my_team'] = player_df['id'].apply(lambda x: 1 if x in my_team_id_list else 0)
    player_df = player_df[player_df.in_my_team != 1] 
    
    # only players in the desired position
    player_df = player_df[player_df.position_abv == replacement_position]
    
    # only players within the replacement budget
    player_df = player_df[player_df.cost_in_mil <= replacement_budget]
    player_df = player_df.drop(['club_limit_exceeded', 'in_my_team'], axis=1)
    return player_df

# Enter the player position_abv, budget to spend, team_abv and a minimal appearance percentage
# pick between using top points or top player score to pick the player

df_to_select_from = player_score_top_players
# df_to_select_from = points_top_players
replacement_position = 'DEF'
replacement_budget = 7
player_team = 'WOL'
min_appearance_percent = .7
min_next_gw_multiplier = .98
min_next_3_gw_multiplier = .98

replacement_player_score_df = transfer_suggestion(df_to_select_from = df_to_select_from,
                                                  replacement_position = replacement_position,
                                                  replacement_budget = replacement_budget,
                                                  player_team = player_team,
                                                  min_appearance_percent = min_appearance_percent
                                                 )
replacement_player_score_df.head(50)

,cost_in_mil,team_abv,position_abv,web_name,id,total_points_inc_gw_0,avg_points_per_app,gw_multi,nxt_gw_pred_score,nxt_3_gw_multi,nxt_3gw_pred_score,player_score_v2,appearance_percentage,net_transfers_this_gameweek,net_transfers_this_season
26,5.2,WHU,DEF,Cresswell,435,49.548387,4.469583,1.082949,4.840331,1.019978,4.558875,0.921825,0.909091,385,668895
34,4.5,WHU,DEF,Masuaku,442,37.529412,3.704431,1.082949,4.011710,1.019978,3.778437,0.882864,0.818182,19,132820
46,5.2,ARS,DEF,Bellerín,11,45.933333,4.004970,1.034632,4.143670,1.028724,4.120009,0.817408,0.909091,42,322416
47,4.9,TOT,DEF,Dier,391,35.894737,3.276397,1.155556,3.786058,1.087873,3.564302,0.815944,0.818182,253,49820
50,6.2,MCI,DEF,Walker,270,42.931034,3.743850,1.093878,4.095313,1.184301,4.433844,0.814604,0.900000,330,446246
57,4.9,WHU,DEF,Ogbonna,432,39.612903,3.540825,1.082949,3.834534,1.019978,3.611563,0.774985,0.909091,98,17614
79,5.6,MCI,DEF,Dias,556,23.000000,2.840085,1.093878,3.106706,1.184301,3.363515,0.684168,0.700000,111,214138
80,4.9,LEI,DEF,Justin,239,37.692308,3.265764,1.021459,3.335844,1.008915,3.294878,0.682636,0.909091,-34,862114
149,5.4,ARS,DEF,Tierney,17,28.800000,2.760335,1.034632,2.855931,1.028724,2.839623,0.542514,0.818182,-22,-440469


In [83]:



# function to create a dictionary from a single players web_name
def create_player_dict(web_name):
    if web_name == '':
        pass
    else:
        return slim_player_df[slim_player_df.web_name == web_name].to_dict('records')[0]
    
    

# manual input variables
min_appearance_percent = .9
min_gw_multiplier = 1
outgoing_player_list = ['Calvert-Lewin','Vestergaard']
outgoing_player_1 = outgoing_player_list[0]
outgoing_player_2 = outgoing_player_list[1]
# outgoing_player_3 = lambda: if len(outgoing_player_list) outgoing_player_list[2]
# outgoing_player_4 = outgoing_player_list[3]

test = lambda x : True if (x > 10 and x < 20) else False

def outgoing_player(player_number):
    print('list length:' + str(len(outgoing_player_list)))
    print('player number' + str(player_number))
    if len(outgoing_player_list) >= player_number:
        outgoing_player_list[player_number-1]
    else:
        pass

print(outgoing_player(1))

    
# calculated variables

# outgoing_player_1_dict = create_player_dict(outgoing_player_1)
# outgoing_player_2_dict = create_player_dict(outgoing_player_2)
# outgoing_player_3_dict = create_player_dict(outgoing_player_3)
# outgoing_player_4_dict = create_player_dict(outgoing_player_4)

# outgoing_player_1_dict

# outgoing_player_1_dict

# type(outgoing_player_1_dict)



# # creating a dataframe for cluds which have reached their limit:

# club_limit_df = my_team_df.groupby(['team_abv']).size().reset_index(name='counts')
# club_limit_df = club_limit_df[club_limit_df.counts >= 3] 
# club_limit_list = list(club_limit_df.team_abv) 


# creating a dataframe for each position

# top_player_df = points_top_players[  (points_top_players.appearance_percentage >= min_appearance_percent)
#                                    & (points_top_players.gw_multi              >= min_gw_multiplier) 
#                                    & (points_top_players.nxt_3_gw_multi        >= min_gw_multiplier)]
                                    
# GKP_top_player_df = top_player_df[top_player_df.position_abv == 'GKP']
# DEF_top_player_df = top_player_df[top_player_df.position_abv == 'DEF']
# MID_top_player_df = top_player_df[top_player_df.position_abv == 'MID']
# FWD_top_player_df = top_player_df[top_player_df.position_abv == 'FWD']


list length:2
player number1
None
